In [2]:
import sqlite3
import pandas as pd

In [3]:
lyrics_db = sqlite3.connect('files/mxm_dataset.db')
track_db = sqlite3.connect('files/track_metadata.db')
compiled_db = sqlite3.connect('files/country_song_project.db')

In [4]:
df = pd.read_csv('files/msd_tagtraum_cd2c.cls', sep="\t", header=None, skiprows=7)
df.columns = ["Track ID", "Genre"]
df_filtered = df[df["Genre"] == "Country"]

In [5]:
df_tracks = pd.read_sql_query("SELECT * FROM songs", track_db)
df_tracks_country = df_tracks[df_tracks["track_id"].isin(df_filtered["Track ID"])]
print(df_tracks_country.shape)
#

(8983, 14)


In [6]:
df_artist_location = pd.read_csv('files/artist_location.txt', sep="<SEP>", header=None)
df_artist_location.columns = ["Artist ID", "X Coordinate", "Y Coordinate", "Artist", "City"]
df_artist_location = df_artist_location.drop(columns=["Artist"])
df_tracks_country = df_tracks_country.merge(df_artist_location, left_on="artist_id", right_on="Artist ID")
df_tracks_country = df_tracks_country.drop(columns=["Artist ID"])
df_tracks_country.head()
#

/var/folders/xj/0wct6fs17lj8c0qthjk1ryb80000gn/T/ipykernel_17866/155918585.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_artist_location = pd.read_csv('files/artist_location.txt', sep="<SEP>", header=None)


,track_id,title,song_id,release,artist_id,artist_mbid,artist_name,duration,artist_familiarity,artist_hotttnesss,year,track_7digitalid,shs_perf,shs_work,X Coordinate,Y Coordinate,City
0,TRMMWEZ128F1455EAC,Foggy Mountain Breakdown,SODSOYL12A6D4F5B39,The Complete Mercury Recordings,ARCUKQ11187B9A9AC9,a9468a9a-b5ea-43b8-85c7-d86542bd2537,Lester Flatt / Earl Scruggs / The Foggy Mounta...,161.95873,0.491050,0.320367,0,227986,-1,0,36.34253,-85.29014,"Overton County, TN"
1,TRWTUUQ128F428237B,Building On Sand,SOHGRTD12A8C1390E3,Foggy Mountain Gospel,ARCUKQ11187B9A9AC9,a9468a9a-b5ea-43b8-85c7-d86542bd2537,Lester Flatt;Earl Scruggs,151.79710,0.583720,0.320367,0,3289083,-1,0,36.34253,-85.29014,"Overton County, TN"
2,TRHYKTA128F4282354,Be Ready For Tomorrow May Never Come,SOGVCPS12A58A7D6B5,Foggy Mountain Gospel,ARCUKQ11187B9A9AC9,a9468a9a-b5ea-43b8-85c7-d86542bd2537,Lester Flatt;Earl Scruggs,160.60036,0.583719,0.320367,0,3289044,-1,0,36.34253,-85.29014,"Overton County, TN"
3,TRRKXZC128F428236C,On The Rock Where Moses Stood,SOJUHEN12A8C1390AC,Foggy Mountain Gospel,ARCUKQ11187B9A9AC9,a9468a9a-b5ea-43b8-85c7-d86542bd2537,Lester Flatt;Earl Scruggs,146.49424,0.583720,0.320367,0,3289068,-1,0,36.34253,-85.29014,"Overton County, TN"
4,TRICUZR12903CE6A45,Baby Blue Eyes,SOZQNUK12AC3DF5694,Foggy Mountain Breakdown,ARCUKQ11187B9A9AC9,a9468a9a-b5ea-43b8-85c7-d86542bd2537,Flatt And Scruggs,179.80036,0.491050,0.320367,0,7603466,-1,0,36.34253,-85.29014,"Overton County, TN"


In [7]:
df_lyric_words = pd.read_sql_query("SELECT * FROM words", lyrics_db)
print(df_lyric_words.shape)

df_reverse_mapping = pd.read_csv('files/mxm_reverse_mapping.txt', sep="<SEP>", header=None)
df_reverse_mapping.columns = ["stemmed", "unstemmed"]
# df_reverse_mapping.head()

df_lyric_words = df_lyric_words.merge(df_reverse_mapping, left_on="word", right_on="stemmed")
df_lyric_words = df_lyric_words.drop(columns=["stemmed"])
df_lyric_words.head()
#
compiled_db.commit()

(5000, 1)


/var/folders/xj/0wct6fs17lj8c0qthjk1ryb80000gn/T/ipykernel_17866/1460977126.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_reverse_mapping = pd.read_csv('files/mxm_reverse_mapping.txt', sep="<SEP>", header=None)


In [19]:
df_lyrics = pd.read_sql_query("SELECT * FROM lyrics", lyrics_db)
print(df_lyrics.shape)
df_lyrics_country = df_lyrics[df_lyrics["track_id"].isin(df_filtered["Track ID"])]
#

(19045332, 5)


In [44]:
df_lyrics_country["str_mult"] = df_lyrics_country.apply(lambda x: ((x["word"] + " ") * x["count"])[:-1], axis=1)
# print(df_lyrics_country.head())
# https://stackoverflow.com/questions/17841149/pandas-groupby-how-to-get-a-union-of-strings
grps = df_lyrics_country.groupby('track_id').agg(
                          bag_lyrics=('str_mult', ' '.join)).reset_index()
df_tracks_country = df_tracks_country.merge(grps, left_on="track_id", right_on="track_id")


/var/folders/xj/0wct6fs17lj8c0qthjk1ryb80000gn/T/ipykernel_17866/3667600348.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lyrics_country["str_mult"] = df_lyrics_country.apply(lambda x: ((x["word"] + " ") * x["count"])[:-1], axis=1)


In [46]:
df_lyric_words.to_sql("words", compiled_db)
df_tracks_country.to_sql("songs", compiled_db)
df_lyrics_country.to_sql("lyrics", compiled_db)

442674

In [ ]:
compiled_db.commit()
lyrics_db.close()
track_db.close()
compiled_db.close()